- grafo cujos nodes são os produtos filtrados na interface por categoria simplificada e rede de fast_food
- tamanho dos nodes é proporcional a % de nutrientes nos produtos
- tamanho das arestas é proporcional a % de nutrientes comuns entre os 2 produtos

In [12]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import pathlib
import getpass

# !pip install pymysql
import pymysql

In [13]:
p = getpass.getpass()
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd = p) 
cur = conn.cursor()

 ·········


In [14]:
cur.execute("USE fast_food;")
cur.execute("SHOW tables;")
for r in cur.fetchall():
   print(r)

('address',)
('category',)
('chain',)
('nutricional_units',)
('product',)
('restaurant',)


In [26]:
# simplificar categorias
cur.execute("SELECT * FROM fast_food.category;") # LIMIT 10
for r in cur.fetchall():
   print(r)

(1, 'WHOPPER')
(2, 'MEAT SANDWICH')
(3, 'CHICKEN SANDWICH')
(4, 'VEGGIES')
(5, 'SIDE DISH')
(6, 'SAUCE')
(7, 'ICE CREAM')
(8, 'SHAKES')
(9, 'BREAKFAST')
(10, 'BEEF & PORK')
(11, 'CHICKEN & FISH')
(12, 'SALADS')
(13, 'SNACKS & SIDES')
(14, 'DESSERTS')
(15, 'BEVERAGES')
(16, 'COFFEE & TEA')
(17, 'SMOOTHIES & SHAKES')
(18, 'CROISSANT')
(19, 'HEALTHY BREAD')
(20, 'BAR AND SHORTBREAD')
(21, 'COOKIE')
(22, 'MUFFIN')
(23, 'FAIRTRADE BANANA')
(24, 'OATMEAL')
(25, 'BREAKFAST SANDWICHES & SAVOURY MUFFINS')
(26, 'TOASTIES')
(27, 'SANDWICHES, PANINIS, WRAPS, ROLLS & FOCACCIAS')
(28, 'HOT BOXES')
(29, 'SALAD BOWLS')
(30, 'COFFEE')
(31, 'CAPPUCCINO')
(32, 'MACCHIATO')
(33, ' DOLCE LATTE')
(34, 'MOCHA')
(35, ' MOCHA')
(36, 'DOUBLESHOT ')
(37, 'TEA')
(38, 'CHAI LATTE')
(39, 'REFRESHERS')
(40, 'SMOOTHIE')
(41, ' BLENDED BEVERAGE')
(42, 'OTHERS BEVERAGE')


In [36]:
cur.execute("SELECT * FROM fast_food.chain;") 
for r in cur.fetchall():
   print(r)

(1, 'BURGER KING')
(2, 'MCDONALDS')
(3, 'STARBUCKS')


In [32]:
cur.execute("DESCRIBE fast_food.nutricional_units;") 
for r in cur.fetchall():
   print(r)

('id_nutricional_units', 'int', 'NO', 'PRI', None, 'auto_increment')
('calories', 'int', 'YES', '', None, '')
('total_fat', 'int', 'YES', '', None, '')
('saturated_fat', 'int', 'YES', '', None, '')
('trans_fat', 'int', 'YES', '', None, '')
('cholesterol', 'int', 'YES', '', None, '')
('sodium', 'int', 'YES', '', None, '')
('carbohydrates', 'int', 'YES', '', None, '')
('dietary_fiber', 'int', 'YES', '', None, '')
('sugars', 'int', 'YES', '', None, '')
('protein', 'int', 'YES', '', None, '')
('vitamin_a', 'int', 'YES', '', None, '')
('vitamin_c', 'int', 'YES', '', None, '')
('calcium', 'int', 'YES', '', None, '')
('iron', 'int', 'YES', '', None, '')
('salt', 'int', 'YES', '', None, '')
('caffeine', 'int', 'YES', '', None, '')


In [34]:
cur.execute("DESCRIBE fast_food.address;") 
for r in cur.fetchall():
   print(r)

('id_address', 'int', 'NO', 'PRI', None, 'auto_increment')
('address', 'varchar(1200)', 'YES', '', None, '')
('city', 'varchar(45)', 'YES', '', None, '')
('state', 'varchar(45)', 'YES', '', None, '')
('post_code', 'varchar(45)', 'YES', '', None, '')
('country', 'varchar(45)', 'YES', '', None, '')


In [44]:
cur.execute("SELECT * FROM fast_food.restaurant LIMIT 5;")
for r in cur.fetchall():
   print(r)

(1, '864-366-2099', 1, 1)
(2, '1 337-329-6352', 2, 1)
(3, '1 337-329-6792', 3, 1)
(4, '208-323-8807', 4, 1)
(5, '208-853-8389', 5, 1)


In [59]:
cur.execute("SELECT * FROM fast_food.product LIMIT 15;")
for r in cur.fetchall():
   print(r)

(1, 'WHOPPER', 1, 1, 1, 361)
(2, 'WHOPPER DUPLO', 1, 2, 1, 456)
(3, 'WHOPPER FURIOSO', 1, 3, 1, 386)
(4, 'WHOPPER JR.', 1, 4, 1, 196)
(5, 'WHOPPER JR. BARBECUE BACON', 1, 5, 1, 249)
(6, 'HAMBÚRGUER', 1, 6, 2, 124)
(7, 'CHEESEBURGER DUPLO', 1, 7, 2, 212)
(8, 'CHEESEBURGER BACON', 1, 8, 2, 144)
(9, 'CHEESEBURGER DUPLO BACON', 1, 9, 2, 235)
(10, 'STACKER DUPLO', 1, 10, 2, 235)
(11, 'STACKER TRIPLO', 1, 11, 2, 486)
(12, 'CHEDDAR', 1, 12, 2, 136)
(13, 'CHEDDAR DUPLO', 1, 13, 2, 181)
(14, 'BIG KING', 1, 14, 2, 312)
(15, 'MEGA STACKER 2.0', 1, 15, 2, 407)


In [48]:
cur.execute("DESCRIBE fast_food.product;")
for r in cur.fetchall():
   print(r)

('id_product', 'int', 'NO', 'PRI', None, 'auto_increment')
('name', 'varchar(500)', 'YES', '', None, '')
('chain_id_chain', 'int', 'NO', 'MUL', None, '')
('nutricional_units_id_nutricional_units', 'int', 'NO', 'MUL', None, '')
('category_id_category', 'int', 'NO', 'MUL', None, '')
('serving_size', 'int', 'YES', '', None, '')


In [57]:
cur.execute("SELECT COUNT(DISTINCT city) , COUNT(DISTINCT state) , COUNT(DISTINCT country) FROM fast_food.address;") 
for r in cur.fetchall():
   print(r)

(3876, 199, 43)
